# 5. Repetition Penalties

We've covered Temperature, Top-K, and Top-P. These manipulate the probabilities of our tokens globally.

However, LLMs still suffer from a known issue: they can get stuck in repetitive loops. If a model generates the word "dog" once, it becomes statistically more likely to generate "dog" again. This can spiral into an infinite loop.

To solve this, we can introduce **Repetition Penalties**. These are applied dynamically to the logits of tokens *that have already appeared* in the generated sequence.

There are two main types:
1. **Frequency Penalty**: Penalizes a token proportional to *how many times* it has appeared.
2. **Presence Penalty**: Penalizes a token if it has appeared *at all*, encouraging the model to introduce completely new topics.

In [1]:
import torch
import torch.nn.functional as F
from collections import Counter
from transformers import AutoModelForCausalLM, AutoTokenizer
import sys
import os

sys.path.append(os.path.abspath('.'))
from metrics import repetition_score, token_diversity_score

device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

model_id = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
model.eval()

Loading weights:   0%|          | 0/290 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2

## Step 1: Tracking Token Frequencies
First, we need a way to count how often a token has appeared in our current generation.

In [2]:
def get_token_counts(input_ids):
    # input_ids is shape (1, seq_length)
    sequence = input_ids[0].tolist()
    return Counter(sequence)

test_prompt = "A cat is a cat is a"
test_ids = tokenizer(test_prompt, return_tensors="pt").input_ids

counts = get_token_counts(test_ids)
print("Token Counts for:", test_prompt)
for token_id, count in counts.items():
    print(f"{repr(tokenizer.decode(token_id))}: {count}")

Token Counts for: A cat is a cat is a
'A': 1
' cat': 2
' is': 2
' a': 2


## Step 2: Applying the Penalties to Logits

Now we write a function that takes our raw logits and our counts, and applies the penalties.

`new_logit = logit - (count * frequency_penalty) - (presence_penalty)`

Note: If the count is 0, the penalties do nothing.

In [3]:
def apply_repetition_penalties(logits, input_ids, freq_penalty, pres_penalty):
    counts = get_token_counts(input_ids)
    
    # Create a copy so we don't modify the original
    penalized_logits = logits.clone()
    
    for token_id, count in counts.items():
        if count > 0:
            # Apply Presence Penalty (flat deduction if count >= 1)
            penalized_logits[token_id] -= pres_penalty
            
            # Apply Frequency Penalty (deduction scaled by count)
            penalized_logits[token_id] -= (count * freq_penalty)
            
    return penalized_logits

## Step 3: Generation with Penalties

Let's put this into a final, robust generation loop. We'll include Temperature, but we'll stick to a greedy choice at the end so we can clearly see the penalties working without randomness muddying the waters.

In [4]:
def penalty_generate(prompt, freq_pen=0.0, pres_pen=0.0, max_new_tokens=30):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    
    print(f"Prompt: '{prompt}' (Freq_Pen={freq_pen}, Pres_Pen={pres_pen})")
    print("Generating: ", end="")
    
    generated_token_ids = []
    
    for _ in range(max_new_tokens):
        with torch.no_grad():
            outputs = model(input_ids)
            
        logits = outputs.logits[0, -1, :]
        
        # 1. Apply Repetition Penalties BEFORE softamx/temperature
        if freq_pen > 0 or pres_pen > 0:
            logits = apply_repetition_penalties(logits, input_ids, freq_pen, pres_pen)
            
        # 2. Greedy choice for this demonstration
        next_token_id = torch.argmax(logits).unsqueeze(0).unsqueeze(0)
        
        generated_token_ids.append(next_token_id.item())
        next_word = tokenizer.decode(next_token_id[0])
        print(next_word, end="", flush=True)
        
        input_ids = torch.cat([input_ids, next_token_id], dim=-1)
        
    print("\n")
    
    # Print diagnostics using our metrics module
    div_score = token_diversity_score(generated_token_ids)
    rep_score = repetition_score(generated_token_ids, n=3)
    print(f"Diagnostics - Diversity: {div_score:.2f} (higher=more unique), 3-Gram Repetition: {rep_score:.2f} (higher=more loops)\n")
    
    return tokenizer.decode(input_ids[0])

# Let's feed it a prompt guaranteed to cause a loop in greedy decoding
loop_prompt = "A cat is a cat is a"
generator_tokens = penalty_generate(loop_prompt, freq_pen=0.0, pres_pen=0.0)

print("\n--- Now let's fix it with a Frequency Penalty ---")
generator_tokens = penalty_generate(loop_prompt, freq_pen=0.5, pres_pen=0.0)

print("\n--- Now let's try extreme Presence Penalty (force vocabulary exploration) ---")
generator_tokens = penalty_generate(loop_prompt, freq_pen=0.0, pres_pen=2.5)


Prompt: 'A cat is a cat is a' (Freq_Pen=0.0, Pres_Pen=0.0)
Generating:  cat is a cat is a cat is a cat is a cat is a cat is a cat is a cat is a cat is a cat is a

Diagnostics - Diversity: 0.10 (higher=more unique), 3-Gram Repetition: 0.89 (higher=more loops)


--- Now let's fix it with a Frequency Penalty ---
Prompt: 'A cat is a cat is a' (Freq_Pen=0.5, Pres_Pen=0.0)
Generating:  dog is a dog is a cat. What is the next sentence?
The answer to this question is:
A dog is a dog.<|endoftext|>Human:

Diagnostics - Diversity: 0.67 (higher=more unique), 3-Gram Repetition: 0.11 (higher=more loops)


--- Now let's try extreme Presence Penalty (force vocabulary exploration) ---
Prompt: 'A cat is a cat is a' (Freq_Pen=0.0, Pres_Pen=2.5)
Generating:  dog.  What type of thing does "cat" and "dog" have in common? Options: - (A) animal - (B)

Diagnostics - Diversity: 0.83 (higher=more unique), 3-Gram Repetition: 0.00 (higher=more loops)



## 🔬 Experimentation Ideas

1. **Feed a prompt that usually causes looping:** (e.g., "Repeat the word cat forever: cat cat cat").
   * *Incrementally increase frequency penalty: 0.0 -> 0.5 -> 1.0 -> 2.0. Watch the diversity score rise.*
2. **Compare the penalties implicitly vs explicitly:**
   * *Frequency Penalty only: What does it output?*
   * *Presence Penalty only: Does it sound different from Frequency Penalty?*
   * *Both combined: Is it too chaotic?*
3. **Measure how token diversity changes:** (unique tokens / total tokens) using `metrics.token_diversity_score`.
   * *Does higher penalty guarantee higher diversity?*
4. **Observe whether penalties reduce coherence at high values:**
   * *If you set `freq_pen=5.0`, does the model start outputting completely random nonsense just to avoid using words it has already used?*

In [5]:
import sys
import os
sys.path.append(os.path.abspath('.'))
from metrics import token_diversity_score
import torch

# Ensure local setup works if run independently
# device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
# model_id = "Qwen/Qwen2.5-0.5B"
# from transformers import AutoModelForCausalLM, AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

print("=== Experiment 1: Incrementally increasing frequency penalty ===")
loop_prompt = "Repeat the word cat forever: cat cat cat"
print(f"\nPrompt: {loop_prompt}")

for freq in [0.0, 0.5, 1.0, 2.0]:
    # We reuse the penalty_generate from above. 
    print(f"\n--- Freq Penalty: {freq} ---")
    penalty_generate(loop_prompt, freq_pen=freq, pres_pen=0.0, max_new_tokens=40)

print("\n=== Experiment 2 & 3: Implicit vs Explicit Penalties & Diversity Score ===")
test_prompt = "A beautiful sunset over the mountains is"
print(f"\nPrompt: {test_prompt}")

print("\n--- Frequency Penalty Only (1.5) ---")
penalty_generate(test_prompt, freq_pen=1.5, pres_pen=0.0, max_new_tokens=40)

print("\n--- Presence Penalty Only (1.5) ---")
penalty_generate(test_prompt, freq_pen=0.0, pres_pen=1.5, max_new_tokens=40)

print("\n--- Both Combined (1.5 / 1.5) ---")
penalty_generate(test_prompt, freq_pen=1.5, pres_pen=1.5, max_new_tokens=40)

print("\n=== Experiment 4: Extreme Penalties (Reduce Coherence) ===")
print("Watch the model start outputting complete nonsense or obscure tokens just to avoid repeating itself!")
penalty_generate(test_prompt, freq_pen=5.0, pres_pen=5.0, max_new_tokens=40)


=== Experiment 1: Incrementally increasing frequency penalty ===

Prompt: Repeat the word cat forever: cat cat cat

--- Freq Penalty: 0.0 ---
Prompt: 'Repeat the word cat forever: cat cat cat' (Freq_Pen=0.0, Pres_Pen=0.0)
Generating:  cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat cat

Diagnostics - Diversity: 0.03 (higher=more unique), 3-Gram Repetition: 0.97 (higher=more loops)


--- Freq Penalty: 0.5 ---
Prompt: 'Repeat the word cat forever: cat cat cat' (Freq_Pen=0.5, Pres_Pen=0.0)
Generating: 
cat cat cat

Repeat the word "cat" forever: cat cat cat

Repeat the word "cat" forever: cat cat cat

Repeat the word "cat" forever: cat<|endoftext|>Human

Diagnostics - Diversity: 0.33 (higher=more unique), 3-Gram Repetition: 0.58 (higher=more loops)


--- Freq Penalty: 1.0 ---
Prompt: 'Repeat the word cat forever: cat cat cat' (Freq_Pen=1.0, Pres_Pen=0.0)
Generating: 
cat

Repeat the

'A beautiful sunset over the mountains is a perfect example of how nature can be so breathtakingly stunning. The colors and shapes are truly mesmerizing, with shades ranging from deep reds to warm oranges.\nThe sun sets in hues that range'